In [1]:
import os
import openai
from dotenv import load_dotenv

# **BURAYI .env DOSYANIZIN YERI OLACAK SEKILDE DEGISTIRIN***
dotenv_path = 'key.env'
is_loaded = load_dotenv(dotenv_path)

# Load config values

# Setting up the deployment name
chatgpt_model_name = os.getenv('CHATGPT_MODEL')

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = os.getenv('OPENAI_API_BASE')

# Currently Chat Completions API have the following versions available: 2023-03-15-preview
openai.api_version = os.getenv('OPENAI_API_VERSION')

deployment_name = os.getenv('DEPLOYMENT_NAME')


print("Are environment variables loaded correctly? ", is_loaded)

Are environment variables loaded correctly?  True


In [1]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("docs\MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs\MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs\MachineLearning-Lecture02.pdf"),
    PyPDFLoader("docs\MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [2]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(docs)
len(splits)

209

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings()

c:\Users\cbark\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [6]:
import numpy as np
np.dot(embedding1, embedding2)
np.dot(embedding1, embedding3)
np.dot(embedding2, embedding3)

0.005847679808422499

In [9]:
#! pip install chromadb
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'
!del -rf ./docs/chroma  # remove old database files if any (change del to rm in linux)
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

Invalid switch - "docs".


418


In [10]:
question = "is there an email i can ask for help"
docs = vectordb.similarity_search(question,k=3)
len(docs)
docs[0].page_content
vectordb.persist()

In [11]:
question = "what did they say about matlab?"
docs = vectordb.similarity_search(question,k=5)

docs[0]
docs[1]

Document(page_content="So later this quarter, we'll use the discussion sections to talk about things like convex \noptimization, to talk a little bit about hidde n Markov models, which is a type of machine \nlearning algorithm for modeling time series and a few other things, so  extensions to the \nmaterials that I'll be covering in the main  lectures. And attend ance at the discussion \nsections is optional, okay?  \nSo that was all I had from l ogistics. Before we move on to start talking a bit about \nmachine learning, let me check what questions you have. Yeah?  \nStudent : [Inaudible] R or something like that?  \nInstructor (Andrew Ng) : Oh, yeah, let's see, right. So our policy has been that you're \nwelcome to use R, but I would strongly advi se against it, mainly because in the last \nproblem set, we actually supply some code th at will run in Octave  but that would be \nsomewhat painful for you to translate into R yourself. So for your other assignments, if \nyou wanna submit 

In [12]:
question = "what did they say about regression in the third lecture?"
docs = vectordb.similarity_search(question,k=5)

for doc in docs:
    print(doc.metadata)

print(docs[4].page_content)

{'page': 2, 'source': 'docs\\MachineLearning-Lecture02.pdf'}
{'page': 2, 'source': 'docs\\MachineLearning-Lecture02.pdf'}
{'page': 0, 'source': 'docs\\MachineLearning-Lecture03.pdf'}
{'page': 0, 'source': 'docs\\MachineLearning-Lecture03.pdf'}
{'page': 7, 'source': 'docs\\MachineLearning-Lecture03.pdf'}
other is, I don’t know, I can also mumble about ju stifications, such as things to the central 
limit theorem. It turns out that if you, for th e vast majority of problems, if you apply a 
linear regression model like this and try to m easure the distribution of the errors, not all 
the time, but very often you find that the erro rs really are Gaussian. That this Gaussian 
model is a good assumption for the error in regression proble ms like these. Some of you 
may have heard of the central limit theo rem, which says that the sum of many 
independent random variables will tend towards a Gaussian. So if the error is caused by many effects, like the mood of th e seller, the mood of the bu